In [34]:
import numpy as np
import pandas as pd

## Task 01: Làm sạch dữ liệu phục vụ cho các công cụ sau 

#### Đọc dữ liệu

In [35]:
shipper_info = pd.read_csv("/kaggle/input/shippper-st/shipper_info.csv")
order_info = pd.read_csv("/kaggle/input/shippper-st/order_info.csv")
customer_rating = pd.read_csv("/kaggle/input/shippper-st/customer_rating.csv")

#### Nhìn tổng quan qua dữ liệu

##### SHIPPER INFO

In [36]:
shipper_info.head()

,shipper_id,full_name,region
0,S100,trAn thi b,bac
1,S101,LE VAN C,bac
2,S102,trAn thi b,Bắc
3,S103,LE VAN C,bac
4,S104,pham thi d,Trung


In [37]:
shipper_info.dtypes

shipper_id    object
full_name     object
region        object
dtype: object

##### XỬ LÝ DỮ LIỆU SHIPPER INFO ( LÀM SẠCH SHIPPER_ID VÀ FULL_NAME, CHUẨN HÓA LẠI REGION )

In [38]:
#Vì có thể sai kiểu nên ép kiểu str và xóa khoảng trắng đầu cuối
shipper_info["shipper_id"] = shipper_info["shipper_id"].astype(str).str.strip()

#Sửa lại định dạng tên ở full_name bằng cách chuyển hết về chữ cái thường rồi viết lại định dạng viết hoa mỗi chữ cái đầu
shipper_info['full_name']=(
    shipper_info['full_name']
    .astype(str)
    .str.strip()
    .str.lower()
    .str.title())

#Chuẩn hóa lại định dạng của cột region
def chuan_hoa(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if "b" in x:
        return "Bắc"
    elif "tr" in x:
        return "Trung"
    elif "n" in x:
        return "Nam"
    else:
        return np.nan

shipper_info["region"] = shipper_info["region"].apply(chuan_hoa)

In [39]:
# Kiểm tra thử kết quả sau khi sửa full_name
shipper_info['full_name'].head()

0    Tran   Thi B
1     Le  Van   C
2    Tran   Thi B
3     Le  Van   C
4    Pham Thi   D
Name: full_name, dtype: object

In [40]:
# Kiểm tra thử kết quả sau khi sửa region
shipper_info['region'].head()
# Vì đã khợp vùng với dữ liệu cũ nên ta chuyển bước tiếp theo

0      Bắc
1      Bắc
2      Bắc
3      Bắc
4    Trung
Name: region, dtype: object

##### ORDER_INFO

In [41]:
order_info.head()

,order_id,shipper_id,distance_km,delivery_time_min
0,OD1000,S117,3.2,45
1,OD1001,S117,-2,20
2,OD1002,S112,7,40m
3,OD1003,S117,-2,25
4,OD1004,S119,1.5,30


In [42]:
order_info.dtypes

order_id             object
shipper_id           object
distance_km          object
delivery_time_min    object
dtype: object

##### XỬ LÝ DỮ LIỆU ORDER INFO ( XỬ LÝ DỮ LIỆU ÂM, KÝ TỰ LẪN SỐ, CHUẨN HÓA DELIVERY_TIME, DỮ LIỆU TRỐNG )

In [43]:
#chuyển dữ liệu sáng kiểu số để ép giá trị tuyệt đối số âm thành số dương
order_info["distance_km"] = pd.to_numeric(
    order_info["distance_km"], errors="coerce"
).abs()

#Ở phần distance_km quan sát em thấy có dữ liệu bao gồm km, và m nên e muốn xóa đi
#Và những phần dữ liệu trống e muốn chèn vào là dữ liệu trung bình của những dữ liệu đã có để tiện cho phân tích
# Xóa ký tự, chỉ giữ lại số 
order_info["delivery_time_min"] = (
    order_info["delivery_time_min"]
    .astype(str)
    .str.extract(r"(-?\d+\.?\d*)")[0]   # trích số
    .astype(float)
)

# Lấy trị tuyệt đối đề phòng có dữ liệu âm
order_info["delivery_time_min"] = order_info["delivery_time_min"].abs()

# Thay NA bằng giá trị trung bình
delivery_time__min_mean = order_info["delivery_time_min"].mean()
order_info["delivery_time_min"] = order_info["delivery_time_min"].fillna(distance_mean)

In [44]:
#Kiểm tra kết quả sau khi làm sạch
order_info.head()
# delivery_time_min đã xóa ký tự và lấp đầy các dòng trống dữ liệu bằng dữ liệu trung bình
# distance_km đã xóa các dữ liệu âm

,order_id,shipper_id,distance_km,delivery_time_min
0,OD1000,S117,3.2,45.0
1,OD1001,S117,2.0,20.0
2,OD1002,S112,7.0,40.0
3,OD1003,S117,2.0,25.0
4,OD1004,S119,1.5,30.0


##### CUSTOMER RATING

In [45]:
customer_rating.head(10)

,order_id,rating
0,OD1020,3
1,OD1032,3
2,OD1001,5*
3,OD1037,4
4,OD1022,NaN
5,OD1054,5*
6,OD1011,NaN
7,OD1012,3
8,OD1017,3
9,OD1041,4


In [46]:
customer_rating.dtypes

order_id    object
rating      object
dtype: object

##### XỬ LÝ DỮ LIỆU ( SAI ĐỊNH DẠNG, DỮ LIỆU TRỐNG VÀ LẤY 5 DÒNG DỮ LIỆU ĐẦU SAU KHI LÀM SẠCH )

In [54]:
# Vì là dữ liệu đánh giá của khách nên nên không thể dùng cách lấy giá trị trung bình 
# Cần dữ liệu thực tế nhất nên ta xóa dữ liệu trống và chỉ phân tích các dữ liệu còn lại
customer_rating["rating"] =  (
    customer_rating["rating"]
    .astype(str)
    .str.extract(r"(-?\d+\.?\d*)")[0]   # trích số
    .astype(float)
)
customer_rating=customer_rating.dropna(subset=['rating']) # Xóa dữ liệu trống chứa NaN

In [57]:
# Kiểm tra thử kết quả sau khi đã làm sạch
customer_rating.head(10)
# Nhận thấy các dữ liệu đã xóa đi ''*'' cho cùng định dạng, xóa các dòng trống đánh giá NaN
# Phân tích giờ sẽ chỉ dựa vào những đánh giá thực tế 

,order_id,rating
0,OD1020,3.0
1,OD1032,3.0
2,OD1001,5.0
3,OD1037,4.0
5,OD1054,5.0
7,OD1012,3.0
8,OD1017,3.0
9,OD1041,4.0
10,OD1000,5.0
11,OD1049,4.0


In [58]:
# Chỉ lấy 5 dòng đầu 1-5
customer_rating.head()

,order_id,rating
0,OD1020,3.0
1,OD1032,3.0
2,OD1001,5.0
3,OD1037,4.0
5,OD1054,5.0


### XUẤT FILE ĐÃ LÀM SẠCH ĐỂ SỬ DỤNG CÁC CÔNG CỤ CÒN LẠI

In [60]:
shipper_info.to_csv("shipper_info_clean.csv", index=False)
order_info.to_csv("order_info_clean.csv", index=False)
customer_rating.to_csv("customer_rating_clean.csv", index=False)